In [1]:
import cv2
from PIL import Image
import os
import numpy as np
import pandas as pd
import requests
import ast
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from hashlib import md5
from multiprocessing import cpu_count

masks_path = '/home/jack/Mounts/DiskOne/kona_coffee/masks'

os.makedirs(masks_path, exist_ok=True)

df = pd.read_csv('annotated.csv')

old_df = None
if os.path.exists('segmented.csv'):
    old_df = pd.read_csv('segmented.csv')

# remove images already segmented
# but remember, there are multiple masks per image
# so we assume the previous segmentation executed successfully
if old_df is not None:
    df = df[~df['image'].isin(old_df['image'])]

df

,image,x,y
1854,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,115,73
1855,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,268,56
4154,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,69,324
4155,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,250,202
4156,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,293,202
...,...,...,...
4750,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,20,227
4751,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,40,165
4752,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,44,62
4753,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,306,23


In [2]:
#url = 'http://172.23.0.100:30280/api/annotate/points'
url = 'http://fast-sam-api.kubby.ninja/api/annotate/points'

image_cache = {}

def annotate(row):
    image_path = row.image
    point = f'{int(row.x)},{int(row.y)}'
    
    if image_path in image_cache:
        imagedata = image_cache[image_path]
    else:
        with open(image_path, 'rb') as f:
            imagedata = f.read()        
        image_cache[image_path] = imagedata
        
    if len(image_cache) > 100:
        del image_cache[list(image_cache.keys())[0]]

    image_hash = md5(imagedata).hexdigest()

    response = None
    for _ in range(3):
        try:
            response = requests.get(
                url=url,
                headers={'x-image-hash': image_hash},
                params={'p': [point]},
                data=imagedata,
                timeout=300,
            )
        except Exception as e:
            print(e)
            continue
        
        if response.status_code != 200:
            continue
        
        break
    
    # If we failed to get a response, return None
    if response is None or response.status_code != 200:
        return
    
    dtype = response.headers['x-numpy-dtype']
    shape = response.headers['x-numpy-shape']

    results = np.frombuffer(response.content, dtype=np.dtype(dtype))
    results = results.reshape(ast.literal_eval(shape))
    result = results[0]
    
    return row, result

In [3]:
pool = ThreadPoolExecutor(6)
data = []

futures = []
for row in df.itertuples():
    futures.append(pool.submit(annotate, row))

for future in tqdm(as_completed(futures), total=len(futures)):
    results = future.result()
    
    if results is None:
        continue
    
    row, mask = results
    
    basename = os.path.basename(row.image).split('.')[0]
    mask_file = os.path.join(masks_path, f'{basename}.{int(row.x)}.{int(row.y)}.mask.npy')
    
    np.save(mask_file, mask)
    data.append([row.image, row.x, row.y, mask_file])

pool.shutdown(wait=True)

  0%|          | 0/603 [00:00<?, ?it/s]

100%|██████████| 603/603 [11:55<00:00,  1.19s/it]


In [4]:

df = pd.DataFrame(data, columns=['image', 'x', 'y', 'mask'])

df = df.dropna()

if old_df is not None:
    df = pd.concat([old_df, df])

df.to_csv('segmented.csv', index=False)

df

,image,x,y,mask
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,308,147,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,273,222,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,324,300,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,272,321,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
...,...,...,...,...
596,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,110,240,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
597,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,21,327,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
598,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,282,134,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
599,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,20,227,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
